In [ ]:

from dotenv import load_dotenv
import requests
from pathlib import Path
import os
import ast
import json
import pandas as pd 

In [ ]:

load_dotenv(dotenv_path=Path.cwd() / ".env")
api_key = os.getenv("LANGTRACE_API_KEY")
project_id = os.getenv("LANGTRACE_PROJECT_ID")

In [ ]:

url = 'https://app.langtrace.ai/api/traces'
headers = {
'x-api-key': api_key,
'Content-Type': 'application/json'
}

payload = {
'page': 1,
'pageSize': 100, # Maximum value is 100.
'projectId': project_id,
}

response = requests.post(url, headers=headers, json=payload)

In [ ]:
test = ast.literal_eval(response.text)['traces']['result']

In [ ]:
spans = []
for item in test:
    if isinstance(item, list):
        spans.extend(item)
    else:
        spans.append(item)

# Parse each span into row dicts
rows = []

for span in spans:
    # Parse attributes (top-level key/values)
    attrs_str = span.get("attributes") or "{}"
    try:
        attrs = json.loads(attrs_str)
    except json.JSONDecodeError:
        attrs = {}

    # Parse events (array of prompt/completion etc.)
    events_str = span.get("events") or "[]"
    try:
        events = json.loads(events_str)
    except json.JSONDecodeError:
        events = []

    prompt_text = None
    completion_text = None

    for e in events:
        name = e.get("name")
        e_attrs = e.get("attributes", {})

        # Prompt text (raw string)
        if name == "gen_ai.content.prompt":
            prompt_text = e_attrs.get("gen_ai.prompt", None)

        # Completion: parse inner JSON and pull assistant content
        if name == "gen_ai.content.completion":
            comp_raw = e_attrs.get("gen_ai.completion")
            if not comp_raw:
                continue

            try:
                messages = json.loads(comp_raw)
                if isinstance(messages, dict):
                    messages = [messages]

                assistant_msgs = [
                    m.get("content", "")
                    for m in messages
                    if m.get("role") == "assistant"
                ]
                if assistant_msgs:
                    # join in case there are multiple
                    completion_text = " ".join(assistant_msgs)
                else:
                    completion_text = comp_raw
            except json.JSONDecodeError:
                # if it's not valid JSON, just keep the raw string
                completion_text = comp_raw

    # Build a row for the DataFrame
    def to_int_or_none(val):
        try:
            return int(val)
        except (TypeError, ValueError):
            return None

    row = {
        "Name": span.get("name"),
        "GenAI System": attrs.get("gen_ai.system"),
        "Request Model": attrs.get("gen_ai.request.model"),
        "Response Model": attrs.get("gen_ai.response.model"),
        "Status": span.get("status_code"),
        "Request Time": span.get("start_time"),
        "Duration_ms": to_int_or_none(span.get("duration")),
        "Input Tokens": to_int_or_none(attrs.get("gen_ai.usage.input_tokens")),
        "Output Tokens": to_int_or_none(attrs.get("gen_ai.usage.output_tokens")),
        "Total Tokens": to_int_or_none(attrs.get("gen_ai.usage.total_tokens")),
        "Prompt": prompt_text,
        "Completion": completion_text,
    }

    rows.append(row)

df = pd.DataFrame(rows)



In [ ]:
df

In [ ]:
df[['Direction', 'Reasoning']] = df['Completion'].str.split(':', n=1, expand=True)

In [ ]:
df